### Preparation

In [1]:
#Load libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from module import * 

In [2]:
#Load data in
tickers = ["TSLA", "AAPL", "AMD"]
start = "2010-01-01"
end = "2024-12-31"

df_prices, df_changes = download_stock_price_data(tickers, start, end)

[*********************100%***********************]  3 of 3 completed


### Signal 1: Moving Average Crossover with MACD Indicator

In [5]:
def signal01(prices, short_ma, long_ma, short_macd, long_macd, signal_window_macd):

    #MA Signal
    ma_sig, _ = ma_signal(prices, short_ma, long_ma)

    #MACD Signal
    macd_sig, _ = signal_macd(prices, short_macd, long_macd, signal_window_macd)

    #Combine Signals
    combined = combine_two_subsignals(ma_sig, macd_sig)

    #Return combined signal dataframe
    signals = pd.DataFrame(index=prices.index)
    signals['signal'] = combined
    signals['position_change'] = signals['signal'].diff().fillna(0)

    return signals

In [7]:
signals = signal01(df_prices['AAPL'], 50, 100, 12, 16, 9)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(signals)

### Signal 2: RSI with Bollinger Bands

### Signal 3: Donchian Channel with ADX